In [505]:
import warnings
warnings.filterwarnings("ignore")

In [506]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pytesseract
import sys
#Setting tesseract path in local
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/Cellar/tesseract/5.1.0/bin/tesseract'

In [507]:
from selenium.webdriver.common.keys import Keys #For accessing buttons
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
import time
import pandas as pd
import math
from urllib.request import urlopen,  Request

#from PIL import Image, ImageOps, ImageEnhance
#from io import StringIO

In [524]:
browser = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 99.0.4844
[WDM] - Get LATEST driver version for 99.0.4844
[WDM] - Driver [/Users/jeyasriramesh/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache


In [509]:
def to_searchpage():
    #Navigating to main frame inside homepage
    home_url = "https://www.sijilat.bh/internet_main.aspx"
    browser.get(home_url)
    time.sleep(2)
    browser.switch_to.frame("frame_main")

    #Closing pop-up
    browser.switch_to.frame(0)
    browser.find_element_by_xpath('//*[@id="bs_LblockPop"]/div/div/a').click()

    #Switch to the immediate parent frame
    browser.switch_to.parent_frame()
    
    #Navigating to Target search page
    browser.execute_script("fn_menu_click('01010000')")
    time.sleep(2)
    #browser.switch_to.frame("frame_main")

In [510]:
def get_captcha():
    
    captcha = ''
    while len(captcha) < 4 : 
        
        #Refresh search_page
        to_searchpage()
        
        #Identify and download captcha
        img_data = browser.find_element_by_id("BodyPlaceHolder_imgCaptcha")
        scrrenshot = img_data.screenshot_as_png
        with open('VDfiles/captcha.png', 'wb') as f:
            f.write(scrrenshot)
    
        #Extract captcha content
        captcha = pytesseract.image_to_string('VDfiles/captcha.png') 
        captcha = captcha.replace(" ", "").strip()
    
    return captcha
    

In [511]:

def check_captcha_text():
    
    #Check if any captcha alert is displayed
    try:
        if (browser.find_element_by_id("btnAlertOK").is_displayed()):
            browser.find_element_by_id("btnAlertOK").click()
            to_searchpage()
            time.sleep(1)
            captcha = get_captcha()
            return captcha
    except:
        return '1'
    

In [523]:
companies = ['HSBC', 'SONY', 'QATAR','ALMAHBOOB','LESHBOONA','UNIVERSAL CATERING','INTERNATIONAL BOUTIQUE']

In [526]:
for company in companies:
   
    #Extracting Captcha from search page
    captcha = get_captcha()
    
    #Commercial Name field id -  BodyPlaceHolder_txtCom
    browser.find_element_by_id("BodyPlaceHolder_txtCom").clear()
    browser.find_element_by_id("BodyPlaceHolder_txtCom").send_keys(company)

    #Enter captcha
    browser.find_element_by_id("BodyPlaceHolder_tb_captcha").clear()
    browser.find_element_by_id("BodyPlaceHolder_tb_captcha").send_keys(captcha)
    
    #click search button
    browser.find_element_by_id('BodyPlaceHolder_btnSearch').click()
    time.sleep(3)
    #Incase of captcha alert
    if len(check_captcha_text()) > 1:
        
        captcha = check_captcha_text()
        
        #Enter captcha
        browser.find_element_by_id("BodyPlaceHolder_tb_captcha").clear()
        browser.find_element_by_id("BodyPlaceHolder_tb_captcha").send_keys(captcha)
    
        #Commercial Name field id -  BodyPlaceHolder_txtCom
        browser.find_element_by_id("BodyPlaceHolder_txtCom").clear()
        browser.find_element_by_id("BodyPlaceHolder_txtCom").send_keys(company)

        #click search button
        browser.find_element_by_id('BodyPlaceHolder_btnSearch').click()
    
    #Collecting records details in a table 
    time.sleep(3)
    df_list = pd.read_html(browser.page_source) # this parses all the tables in webpages to a list
    df = df_list[1]

    #Getting 5 CR_NO records
    for i in range(1):
        cr_no = str(df.iloc[i][0]).split("-")
        
        if i > 0:
            browser.execute_script("window.history.go(-1)")
            time.sleep(2)
        
        #Navigating to company details by record
        browser.execute_script("f_MoveLoading(this);f_CRDetailPage("+ cr_no[0] +"," + cr_no[1]+ ");")
        
        #Saving screenshot
        filename = company + "_" + str(cr_no[0])+ "_" + str(cr_no[1])
        file_path = 'VDfiles/screenshot/'+filename+'.png'
        
        #Getting a screenshot
        time.sleep(3)
        browser.execute_script("document.body.style.zoom='50%'") 
        time.sleep(1)
        browser.find_element_by_tag_name('body').screenshot(file_path)
        
        #Enhancing screenshot

        #box = (450, 220, 900, 600)
        #im = Image.open(file_path)
        #rgb_im = im.convert('RGB')
        #region = rgb_im.crop(box)
        #enhancer = ImageEnhance.Sharpness(im)
        #enhanced_im = enhancer.enhance(1.0)
        #r = ImageOps.invert(region.convert('L').point(lambda x: 255 if x > 254 else 0))
        #enhanced_im.save('test.png',optimize=False, quality=99)
        #region.save('screen.jpg', 'JPEG', optimize=False, quality=99)

        #Converting company details into a dataframe
        time.sleep(2)
        
        try: 
            #browser.switch_to.frame("frame_main")
            df_comp_list = pd.read_html(browser.page_source) # this parses all the tables in webpages to a list
            df_comp = df_comp_list[1]
            df_comp = df_comp[1:4]


            #Formatting as per requirement
            df_comp = df_comp.replace('Nationality', 'Country')

            #print(df_comp)

            #Converting contents to a dictionary (i.e key value pairs)
            key_pair = [{i:j} for i, j in df_comp.iloc[:,[2,3]].values]
            final_key_pair = key_pair[:len(key_pair)-2]
            final_key_pair.extend([{i:j} for i, j in df_comp.iloc[:,[0,1]].values])
            final_key_pair

            #print(final_key_pair)

            #writing the contents to a text file
            filename_text = company + "_" + str(cr_no[0])+ "_" + str(cr_no[1])
            file_path_text = 'VDfiles/keypairs_file/'+filename_text+'.txt'
            with open(file_path_text, 'w') as f:
                print(final_key_pair, file=f)
        except:
            pass
 